# Prepairing chesscom data to use

Working with API, prepairing good forma etc

## Imports and functions

In [6]:
# coding: utf-8

# our all
import numpy as np
import pandas as pd

# usefull pandas settings
pd.set_option('display.max_rows', 45000)
pd.set_option('display.max_columns', 50000)
pd.set_option('display.max_colwidth', 5000)

# for API working and current time
import requests
import datetime
import json

# chess pgn-reading tool
from pgn_parser import pgn, parser

# multistreaming
import threading

# for file working
from os import listdir
from os.path import isfile, join

# own libs
import sys
sys.path.append('//home//roman_vm//my_lib//')

from data_load import *

In [7]:
# heplfull instruments
# convert unixtime to time:
# fr_unixtime(1565211491) -> '2019-08-07 20:58:11'
def fr_unixtime(ts):
    from datetime import datetime
    return datetime.utcfromtimestamp(int(ts)).strftime('%Y-%m-%d %H:%M:%S')

# convert integer month number to string API format:
# 1 -> '01' 
# 9 -> '09' 
# 12 > '12'
# 112 -> -1
def get_number(x):
    if x>=10 and x<100:
        return str(x)
    elif x<10:
        return '0'+str(x)
    else:
        return -1
    
# retunr classic Elo propabilities
# elo_prob(2882, 2722) -> 0.7152 (72% chanses Carlsen (2882) to beat Wan Hao (2722))
def elo_prob(rw, rb):
    rw=float(rw)
    rb=float(rb)
    res=1/(1+np.power(10, (rb-rw)/400))
    return res

# working with pgn
# pip install pgn-parser
def get_pgn(text):
    from pgn_parser import pgn, parser
    try:
        game = parser.parse(text, actions=pgn.Actions())
        try:
            score=game.tag_pairs['Result']
        except:
            score='Unknown'  
        try:
            date=game.tag_pairs['Date']
        except:
            date='Unknown'
        try:
            event=game.tag_pairs['Event']
        except:
            event='Unknown'
        try:
            time=game.tag_pairs['UTCTime']
        except:
            time='Unknown'
        try:
            eco=game.tag_pairs['ECO']
        except:
            eco='Unknown'
        try:
            ecourl=game.tag_pairs['ECOUrl']
        except:
            ecourl='Unknown'
        
        res={'score':score, 
                'date': date, 
                'event': event,
                'time':time, 
                'ECO':eco, 
                'ECO_url':ecourl}
        
    except:
        res={'score':'Unknown',
                'date': '1970-01-01', 
                'time':'00:00:00',
                'event': 'Unknown', 
                'ECO':'Unknown' ,
                'ECO_url':'Unknown'}
    return res

# functions for predictions
# find col with target user (white or black)
def find_col(target_user, col1, wh_val, bl_val):
    if col1==target_user:
        return wh_val
    else:
        return bl_val
# reverse fun of find_col    
def find_opp(target_user, col1, wh_val, bl_val):
    if col1!=target_user:
        return wh_val
    else:
        return bl_val

In [8]:
get_pgn('[Event "Live Chess"]\n[Site "Chess.com"]\n[Date "2020.04.01"]\n[Round "-"]\n[White "Rosolimo"]\n[Black "Raghavdutt"]\n[Result "0-1"]\n[ECO "E61"]\n[ECOUrl "https://www.chess.com/openings/Kings-Indian-Defense-Normal-Variation"]\n[CurrentPosition "8/pp5k/6pp/2pP4/1P1b3q/P7/4nPQ1/3R1RK1 w - -"]\n[Timezone "UTC"]\n[UTCDate "2020.04.01"]\n[UTCTime "10:30:08"]\n[WhiteElo "1575"]\n[BlackElo "1563"]\n[TimeControl "180+2"]\n[Termination "Raghavdutt won by checkmate"]\n[StartTime "10:30:08"]\n[EndDate "2020.04.01"]\n[EndTime "10:36:00"]\n[Link "https://www.chess.com/live/game/4664567570"]\n\n1. d4 {[%clk 0:03:01.9]} 1... g6 {[%clk 0:02:59.4]} 2. c4 {[%clk 0:03:03]} 2... Bg7 {[%clk 0:03:00.8]} 3. Nc3 {[%clk 0:03:04.1]} 3... Nf6 {[%clk 0:03:02.2]} 4. e4 {[%clk 0:03:05.3]} 4... d6 {[%clk 0:03:00.3]} 5. Nf3 {[%clk 0:03:05.6]} 5... O-O {[%clk 0:02:59.3]} 6. Bd3 {[%clk 0:03:06.4]} 6... Na6 {[%clk 0:02:57]} 7. a3 {[%clk 0:03:06.2]} 7... c5 {[%clk 0:02:57.3]} 8. d5 {[%clk 0:03:06.6]} 8... e6 {[%clk 0:02:48.4]} 9. O-O {[%clk 0:03:06.5]} 9... h6 {[%clk 0:02:45.2]} 10. Be3 {[%clk 0:03:06.2]} 10... Ng4 {[%clk 0:02:39.5]} 11. Bf4 {[%clk 0:03:05.2]} 11... exd5 {[%clk 0:02:12.7]} 12. cxd5 {[%clk 0:03:05.2]} 12... Nc7 {[%clk 0:02:01.9]} 13. h3 {[%clk 0:03:04.8]} 13... Nf6 {[%clk 0:01:59]} 14. Nb5 {[%clk 0:03:05]} 14... Nxb5 {[%clk 0:01:58.7]} 15. Bxb5 {[%clk 0:03:03]} 15... Nh5 {[%clk 0:01:59.1]} 16. Bh2 {[%clk 0:03:01.7]} 16... f5 {[%clk 0:02:00]} 17. Bd3 {[%clk 0:02:52.4]} 17... fxe4 {[%clk 0:01:41.2]} 18. Bxe4 {[%clk 0:02:52]} 18... Kh7 {[%clk 0:01:42.4]} 19. Nh4 {[%clk 0:02:51.9]} 19... Qxh4 {[%clk 0:01:22.2]} 20. Bxd6 {[%clk 0:02:41.6]} 20... Bxh3 {[%clk 0:01:08.2]} 21. Bxf8 {[%clk 0:02:33.5]} 21... Rxf8 {[%clk 0:01:08.9]} 22. Qc2 {[%clk 0:02:28.2]} 22... Bf5 {[%clk 0:00:39.9]} 23. Bxf5 {[%clk 0:02:24.5]} 23... Rxf5 {[%clk 0:00:40.9]} 24. Rad1 {[%clk 0:02:21.2]} 24... Bd4 {[%clk 0:00:37.3]} 25. b4 {[%clk 0:02:13]} 25... Rg5 {[%clk 0:00:30.4]} 26. Qd3 {[%clk 0:01:57.4]} 26... Nf4 {[%clk 0:00:21.8]} 27. Qf3 {[%clk 0:01:54.2]} 27... Rxg2+ {[%clk 0:00:18.6]} 28. Qxg2 {[%clk 0:01:55.3]} 28... Ne2# {[%clk 0:00:19.1]} 0-1')

{'ECO': 'E61',
 'ECO_url': 'https://www.chess.com/openings/Kings-Indian-Defense-Normal-Variation',
 'date': '2020.04.01',
 'event': 'Live Chess',
 'score': '0-1',
 'time': '10:30:08'}

In [9]:
# get API data and prepairint it

# get json form site and return string
# exmple get_api_data_to_str('eric', '2014', '01')
def get_api_data_to_str(player, year, month):
    import requests
    response = requests.get('https://api.chess.com/pub/player/'+player+'/games/'+year+'/'+month)
    st=response.text
    return st, response.status_code

# convert api data to pandas for next working
def get_str_data_to_pandas(strng):
    import json
    try:
        js = json.loads(strng)
        df=pd.DataFrame(js)
    except:
        df=pd.DataFrame()      
    return df

# convert multilevel json to table
def prepair_pandas_multilevel_data(df):
    num=len(df)
    new_df=pd.DataFrame()
    for i in range(num):
        b=pd.io.json.json_normalize(df.values[i])
        new_df=pd.concat([new_df, b])
    return new_df 

# get player games in month with preparation
def get_pl_stat(df):    
    # chess result dictionary
    res_dict={
    'win': 1.0,
    'checkmated': 0.0,
    'agreed': 0.5,
    'repetition': 0.5,
    'timeout': 0.0,
    'resigned': 0.0,
    'stalemate': 0.5,
    'lose': 0.0,
    'insufficient': 0.5,
    '50move': 0.5,
    'abandoned': 0.0,
    'kingofthehill': 0.5,
    'threecheck': 0.5,
    'timevsinsufficient': 0.5,
    'bughousepartnerlose': 0.0
    }

    # number of games
    num=len(df)
    
    # cut long and unusefull columns
    df=df[['rules', 'time_class', 'time_control', 'rated',
        'white.@id', 'white.rating', 'white.result', 'white.username',
         'black.@id', 'black.rating', 'black.result', 'black.username',
         'end_time', 'pgn', 
         'url']]
    # score 0.0, 0.5 or 1.0
    df['white.score']=df['white.result'].map(res_dict)
    df['black.score']=df['black.result'].map(res_dict)
    
#     # pgn - long string
    df['pgn']=df['pgn'].apply(get_pgn)
    # result - string like '1-0'
    df['result']=df['pgn'].apply(lambda x: x['score'])
    
    df['event']=df['pgn'].apply(lambda x: x['event'])
    
    df['date']=df['pgn'].apply(lambda x: x['date'])
    df['time']=df['pgn'].apply(lambda x: x['time'])
    
    # ECO - string like 'B10'
    df['eco']=df['pgn'].apply(lambda x: x['ECO'])
    # ECOurl - url like 'https://www.chess.com/openings/B10-Caro-Kann-Defense-2.Nf3-d5'
    df['eco_url']=df['pgn'].apply(lambda x: x['ECO_url'])
    
#     propability from classic formula
    df['white_elo_forecast']=df[['white.rating', 'black.rating']].apply(lambda x: elo_prob(*x), axis=1)
    df['black_elo_forecast']=df[['black.rating', 'white.rating']].apply(lambda x: elo_prob(*x), axis=1)
    
    # after it np.sum(df['game']) means number of games
    df['game']=1
    df['date'] = df['date'].astype('datetime64[ns]')
    # period - stirng like '2019-08'
    df['period']=df['date'].dt.year.astype('str')+'-'+df['date'].dt.month.astype('str')
        
    df.columns=['rules', 'time_class', 'time_control', 'rated', 'white_url',
       'white.rating', 'white.result', 'white.username', 'black_url',
       'black.rating', 'black.result', 'black.username', 'end_time', 'pgn', 'game_url',
       'white.score', 'black.score', 'result', 'event', 'date', 'time', 'eco',
       'eco_url', 'white_elo_forecast', 'black_elo_forecast',
       'game', 'period']
    df=df[[
        'rules', 'time_class', 'time_control', 'rated', 'game',
        'result', 'date', 'time', 'period',
        'white.username', 'black.username',
        'white.rating', 'black.rating', 
        'white.result', 'black.result', 
        'white.score', 'black.score', 
        'event',
        'white_elo_forecast', 'black_elo_forecast',
        'eco', 'eco_url', 
        'game_url', 'white_url', 'black_url'
    ]]
    
    
    df['date'] = df['date'].astype('datetime64[ns]')
    player=df['white.username'].value_counts().index[0]
    year=df['date'].value_counts().index[0].strftime("%Y")
    month=df['date'].value_counts().index[0].strftime("%m")
    return df

# user-based dataframe
def learn_prepair(df, target_user):
    df=df[(df['white.username']==target_user)|(df['black.username']==target_user)]
    df['target_user']=target_user
    df['opponent']=df[['target_user', 'white.username', 'white.username', 'black.username']].apply(lambda x: find_opp(*x), axis=1)
    df['score']=df[['target_user', 'white.username', 'white.score', 'black.score']].apply(lambda x: find_col(*x), axis=1)
    df['rating']=df[['target_user','white.username', 'white.rating', 'black.rating']].apply(lambda x: find_col(*x), axis=1)
    df['opponent_rating']=df[['target_user', 'white.username', 'white.rating', 'black.rating']].apply(lambda x: find_opp(*x), axis=1)
    df['result_reason']=df[['target_user','white.username', 'white.result', 'black.result']].apply(lambda x: find_col(*x), axis=1)
    df['opponent_result_reason']=df[['target_user', 'white.username', 'white.result', 'black.result']].apply(lambda x: find_opp(*x), axis=1)
    df['base_elo_forec']=df[['target_user', 'white.username', 'white_elo_forecast', 'black_elo_forecast']].apply(lambda x: find_col(*x), axis=1)
    df['color']=np.where(df['white.username']==target_user, 'w', 'b')
    df['datime']=(df['date'].astype('str')+' '+df['time'].astype('str')).astype('datetime64[ns]')

    df=df[[
        'target_user', 'opponent', 
        'datime', 'event', 'rules',
        'time_class', 'time_control', 'rated',
        'result', 'score', 'result_reason', 'opponent_result_reason', 
        'color', 'rating', 'opponent_rating', 'base_elo_forec', 'eco',
        'game_url', 'eco_url'
         ]]
    
    return df

# construct all loading steps in one process
def get_df(player, year, month):
    response_string, response_status_code=get_api_data_to_str(player, year, month)
    dirty_df=get_str_data_to_pandas(response_string)
    normalised_df=prepair_pandas_multilevel_data(dirty_df)
    df_with_pgn_data=get_pl_stat(normalised_df)
    user_based_df=learn_prepair(df_with_pgn_data, player)
    
    return user_based_df

# insert data in sql table
def insert_df(df, table_name):
    import datetime 
    now=datetime.datetime.now() 
    df['inserted_at']=now
    
    postgresql_engine=get_engine()
    
    df.to_sql(table_name, con=postgresql_engine, if_exists='append')#, index_label='index')    
    print('Ready: ', len(df), ' rows inserted')   
#     return df
    
# all processes in one fun    
def main_process(user_list, year_range):
    for user in user_list:
        # status to visulise
        print('Loading of games from user ', user, ' started')
        for year in year_range:
            print('Year ', year, ' is beginning')
            for month in range(1,13):
                now=datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                print(now, ' : month ', month, ' started')
                try:
                    insert_df(get_df(user,str(year), get_number(month)), 'chesscom_players')
                except:
                    print(now, 'Error')
                            
# get players list by countries
def get_county_players_list(country):
    import requests
    response = requests.get('https://api.chess.com/pub/country/'+country+'/players')
    st=response.text
    
    return st

In [10]:
def recurse_players_list(where_str):
    opps=get_data("""
    SELECT distinct opponent
    FROM chesscom_players"""+where_str
                 )
    
    opps=opps['opponent'].values
    opps=list(opps)
    
    return opps

In [11]:
# pl=recurse_players_list("""where rated='True' and datime>='2020-01-01'""")
# pl=recurse_players_list("""""")

ERROR:root:An unexpected error occurred while tokenizing input
The following traceback may be corrupted or invalid
The error message is: ('EOF in multi-line string', (1, 25))



UndefinedTable: relation "chesscom_players" does not exist
LINE 3:     FROM chesscom_players
                 ^


In [ ]:
# # for tests
# player='Rosolimo'

# # year='2020'
# # month='04'

# year=2020
# month=4

# # # response_string, response_status_code=get_api_data_to_str(player, year, month)
# # # dirty_df=get_str_data_to_pandas(response_string)
# # # normalised_df=prepair_pandas_multilevel_data(dirty_df)
# # # user_based_df=learn_prepair(df_with_pgn_data, player)

# op=get_df(player,str(year), get_number(month))
# insert_df(op, 'chesscom_players')

## Start working

In [ ]:
year_range=range(2013, 2021)
user_list=['Rosolimo']
# user_list=recurse_players_list(""" where rated='True' and datime>='2020-07-01'""")
main_process(user_list, year_range)

In [ ]:
get_data("""
--drop table chesscom_players


select * from chesscom_players limit 5
""")